<div id="instructions" style="border-radius: 5px; background-color:#f5f5f5;" >
<h1>Instructions</h1>
<p>Look for the <b>2 Your Turn</b> sections to complete the code and/or answer questions.<p>
</div>

# Lesson 5 - Bootstrap and Resample Methods

## Introduction

Bootstrap and resampling are widely applicable statistical methods which relax many of the assumptions of classical statistics. Resampling methods draw heavily on the CLT. Specifically resampling methods:

- Allow computation of statistics from limited data
- Compute statistics from multiple subsamples of dataset
- Minimal distribution assumptions
- Computationally intensive

Commonly used resampling methods include:

- Randomization or permutation methods (e.g., Fisher's exact test)
- Bootstrap: resample with equivalent size and replacement
- Jackknife: leave one out resampling
- Cross validation: resample into folds without replacement


## Pitfalls

But, there is no magic involved. When using resampling methods always keep in mind the several pitfalls:
- If sample is biased, the resample statistic is biased.
- Sample variance and CIs are no better than what the sample allows.


## History

### Randomization and permutation methods

Randomization and permutation methods were pioneered by Fisher as early as 1911. Fisher fully developed the theory in his 1935 book. Scalability of these methods remain limited, even with modern computers. 

### Cross-validation

Cross-validation was originally proposed by Kurtz in 1948. Mosier extended the method to double cross validation in 1951. The modern method of nested or multicross-validation were introduced by Krus and Fuller in 1982. Today, cross-validation is widely used in the testing of machine learning models. 

### Jackknife Methods

Maurice Quenouille originally suggested this method in 1949. The jackknife was fully developed
by John W. Tukey, who gave the method its name, in 1958. Tukey saw that method as a simple tool useful for many purposes like a pocket knife. 


### Bootstrap 

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/Efron1980.jpg" style="float: left; height: 150px; padding-left: 10px;" title="Efron's 1980 book." />
The bootstrap method was first suggested by Efron and Hinkley in 1978 and further developed by Efron in 1979. A full treatment was provided in Efron's 1980 book. 

With increased computing power, use of bootstrap methods continues to expand. Check out <a title="YouTube: Interview with Bradley Efron, BBVA Foundation Frontiers of Knowledge Award in Basic Sciences" href="https://www.youtube.com/embed/H2tOhMaXWvI" target="newwindow">Efron himself explaining</a> the usefulness of bootstrap method.

## Overview of the Bootstrap

The bootstrap method:

- Re-compute statistic many times with sample 
- Randomly sample (e.g., Bernoulli sample) data with replacement
- Subsamples have the same size as original sample
- Works with any statistic … in principle

For example, you can compute the bootstrap mean as:

$$Meanboot = \frac{\Sigma_i mean(sample_i)}{nsample}\\
where,\ for\ example\ with\ 10\ samples,\\
sample_i = X_1 + X_2 + X_3 + X_4 + X_5 + X_6 + X_7 + X_8 + X_1 + X_5$$

### Bootstrap Example

Let's try a first example. You will bootstrap the means of the heights of different populations from Galton's height data. This is an example of a parametric bootstrap estimate. Parametric because our model has a parameter, the mean, we are trying to estimate. 

As a first step, the code in the cell below divides the data set by male and female adult children, and then plots the distributions with the means of the two populations. Run this code and examine the results.

In [ ]:
import pandas
import numpy
from matplotlib import pyplot
import seaborn

%matplotlib inline

In [ ]:
families = pandas.read_csv('https://library.startlearninglabs.uw.edu/DATASCI410/Datasets/GaltonFamilies.csv', 
                            index_col=0)
families.head()

You can select a subset of rows from a dataframe that match a given condition like so (the condition is gender in this case):

In [ ]:
male = families[families.gender == 'male']
female = families[families.gender == 'female']

len(male), len(female)

Note that we can get the min or max of a series of values within a 
dataframe like so:

In [ ]:
families.childHeight.min()

In [ ]:
def plot_hist(x, p=5):
    # Plot the distribution and mark the mean
    pyplot.hist(x, alpha=.5)
    pyplot.axvline(x.mean())
    # 95% confidence interval    
    pyplot.axvline(numpy.percentile(x, p/2.), color='red', linewidth=3)
    pyplot.axvline(numpy.percentile(x, 100-p/2.), color='red', linewidth=3)
    
def plot_dists(a, b, nbins, a_label='pop_A', b_label='pop_B', p=5):
    # Create a single sequence of bins to be shared across both
    # distribution plots for visualization consistency.
    combined = pandas.concat([a, b])
    breaks = numpy.linspace(
        combined.min(), 
        combined.max(), 
        num=nbins+1)

    pyplot.subplot(2, 1, 1)
    plot_hist(a)
    pyplot.title(a_label)
    
    pyplot.subplot(2, 1, 2)
    plot_hist(b)
    pyplot.title(b_label)
    
    pyplot.tight_layout()
    

plot_dists(male.childHeight, female.childHeight, 20, a_label='sons', b_label='daughters')

You can see that the distributions of the heights of the sons and daughters overlap. But, are the means significantly different?

### Bootstrap the Means

Pandas has built-in support for generating bootstrap samples from a given dataframe. We'll use the `.sample()` method of our two dataframes to draw a single bootstrap sample like so:

In [ ]:
female.sample(frac=1, replace=True).head()

We can take the mean of our bootstrap sample in the same way as our full dataset dataframe:

In [ ]:
female.sample(frac=1, replace=True).childHeight.mean()

Now we'll take many (`n_replicas`) bootstrap samples and plot the distribution of sample means as well as the mean of the sample means.

In [ ]:
n_replicas = 1000

female_bootstrap_means = pandas.Series([
    female.sample(frac=1, replace=True).childHeight.mean()
    for i in range(n_replicas)])

male_bootstrap_means = pandas.Series([
        male.sample(frac=1, replace=True).childHeight.mean()
        for i in range(n_replicas)])

plot_dists(male_bootstrap_means, female_bootstrap_means, 
           nbins=80, a_label='sons', b_label='daughters')

The distribution of the bootstrap means do not overlap at all. Evidently, the **difference is significant**. 

## Your Turn 1
Compute and plot the bootstrap distribution of the mean of the heights of the fathers for male children. Compare this estimate to the bootstrap estimate of the mean of the heights of the sons (already computed). Are these means significantly different? 

In [ ]:
#Replace the ???
father_bootstrap_means = pandas.Series([
    ???.sample(frac=1, replace=True).???.mean()
    for i in range(n_replicas)])

#Replace the ???
plot_dists(???, ???, 
           nbins=80, a_label='???', b_label='???')

### Bootstrap Difference in Means

In the previous example you computed and compared two bootstrap distributions of the means from the height data. It is actually quite easy to bootstrap a statistic such as the difference in means. 

In the cell below, we generate bootstrap samples from the full male+female dataset and then difference in the means of male and female `childHeight` for each sample, generating a distribution of sample mean `diffs`. 

Run the code and examine the plotted results.

In [ ]:
diffs = []
for i in range(n_replicas):
    sample = families.sample(frac=1.0, replace=True)
    male_sample_mean = sample[sample.gender == 'male'].childHeight.mean()
    female_sample_mean = sample[sample.gender == 'female'].childHeight.mean()
    diffs.append(male_sample_mean - female_sample_mean)
diffs = pandas.Series(diffs)

plot_hist(diffs)

The distribution of the **difference in means is far from zero**. As before, we can infer that the means of the two populations are **significantly different**. 

But, is the distribution of the difference in means normal, as implied by the CLT? Run the code in the cell below to create a Q-Q normal plot of the bootstrap distribution of the difference in means and examine the results. 

In [ ]:
import statsmodels.api as sm

In [ ]:
## Is the bootstrapped distribution Normal?
fig = sm.qqplot(diffs, line='s')
pyplot.title('Quantiles of standard Normal vs. bookstrapped mean')

The points on the Q-Q Normal plot are nearly on a strait line. Apparently, the bootstrap distribution of the difference in means does conform to the CLT.

## Your Turn 2
In the cell below, create and run the code to compare the difference in means of adult sons to their fathers. Is the difference significant? Also, check if the distribution in the difference in means is approximately normal. 

In [ ]:
# Replace the ???
diffs = []
for i in range(n_replicas):
    sample = families.sample(frac=1.0, replace=True)
    male_sample_mean = sample[sample.gender == '???'].???.mean()
    father_sample_mean = sample[sample.gender == '???'].???.mean()
    diffs.append(male_sample_mean - father_sample_mean)
diffs = pandas.Series(diffs)

plot_hist(diffs)

fig = sm.qqplot(diffs, line='s')
pyplot.title('Quantiles of standard Normal vs. bootstrapped mean')

## Bootstraping Other Statistics

The bootstrap method can be applied to most any statistic. The code in the cell below computes the difference in **medians** of the sons and the fathers. The median is an order statistic, and the values of a median are discretized by the quanta of the samples. Run this code and examine the results.

In [ ]:
median_diffs = []
for i in range(n_replicas):
    sample = families.sample(frac=1.0, replace=True)
    male_sample_median = sample[sample.gender == 'male'].childHeight.median()
    father_sample_median = sample[sample.gender == 'male'].father.median()
    median_diffs.append(male_sample_median - father_sample_median)
median_diffs = pandas.Series(median_diffs)

plot_hist(median_diffs)

The chart shows that the bootstrap distribution of the difference in medians. The **confidence interval overlaps zero**, so we must accept the null hypothesis that the difference in medians is zero.

You can also plot the Q-Q Normal plot of the bootstrap distribution of the difference in medians. Run the code in the cell below and examine the result.

In [ ]:
## Is the bootstrapped distribution Normal?
fig = sm.qqplot(median_diffs, line='s')
pyplot.title('Quantiles of standard Normal vs. bootstrapped median')

Keep in mind that the **CLT only applies to sample means**, not medians or any other statistic. None the less, you can see that the Q-Q normal plot of the bootstrap distribution of the difference in medians tends down the center diagonal of the plot. The zig-zag behavior arises from the quantization inherent in the median.

## The Jackknife

The jackknife is another resampling method. The jackknife is related to the bootstrap, but is a bit more restrictive in its application. Specifically, the jackknife:

- Re-computes a statistic N times with leave-one-out sample, where N is the sample size.
- Uses Random leave one (or n) out sampling.
- Only works for statistics with continuous derivatives.

For example, you can compute the bootstrap mean as:

$$MeanJK = \frac{\Sigma_i mean(sample_i)}{nsample}\\
where,\ for\ example\ with\ 10\ samples,\\
sample_i = X_1 + X_2 + X_3 + X_4 + X_5 + X_6 + X_8 + X_9 + X_{10}$$

A simple example of using a jackknife estimation of a mean is shown in the cell below. Run this code, examine the results and compare them to those obtained with the bootstrap. 

In [ ]:
print("We have %d male rows and %d female rows" % (len(male), len(female)))

We'll use the index of each row within a set (either `male` or `female`) to leave each of the observations (rows) out, one at a time.

In [ ]:
# Re-number the rows in each set starting from zero or else the row indices
# would still match the original `families` dataframe we started with.
male = male.reset_index(drop=True)
female = female.reset_index(drop=True)

Now we can drop any particular row(s) we like and compute our jackknife sample mean on the remaining rows.

One trick to selecting all rows except for a given row index is the following, where we drop the row at index==3 (note the left-most column values skip 3).

In [ ]:
male.drop(male.iloc[[3]].index).head()

We can then compute a summary statistic of our choice on the rows that were not dropped like so:

In [ ]:
male.drop(male.iloc[[3]].index).childHeight.mean()

Let's now use this strategy to leave each of the rows out, one at a time, and collect the summary statistics (child height mean) for each.

In [ ]:
male_jk_means = pandas.Series([
        male.drop(male.iloc[[i]].index).childHeight.mean()
        for i in range(len(male))])

male_jk_means.describe()

We can do the same for the gender==female subset of our data

In [ ]:
female_jk_means = pandas.Series([
        female.drop(male.iloc[[i]].index).childHeight.mean()
        for i in range(len(female))])

female_jk_means.describe()

## Cross Validation

Cross validation is a leave-many-out resampling technique. Cross validation uses multiple non-overlapping subsamples to train and test a data set. Basic cross validation uses the following steps:

- Divide dataset into N subsamples 
- N – 1 Folds train model 
- One Fold evaluate model

The basic concept of cross validation is illustrated below. 

<img src="https://library.startlearninglabs.uw.edu/DATASCI410/img/CrossValidation.jpg" style="height: 200px;" title="Cross validation illustrated" />

Model performance is performed by looking at aggregated summary statistics across the the trained models using the test data for each resample. The resampling reduces bias in the model performance statistics. Most importantly, cross validation reduces unpleasant surprises when a model is placed in production. 

The jackknife is an extreme end member of the family of cross validation methods. With the jackknife, the number of folds equals the number of data points. 

There are several variations on cross validation. The most commonly used in **nest cross validation**. In nested cross validation, cross validation is performed on one or more models to compare performance.



<div id="reminder" style="border-radius: 5px; background-color:#f5f5f5;" >
<h3>Reminder</h3>
<p>Use this notebook to answer the quiz questions related to the <b>Your Turn</b> sections.<p>
</div>